In [6]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

Web Search Tool not so cheap. it cost 1 cents per call.</br>Use it wisely.</br>For more information see https://platform.openai.com/docs/pricing#web-search

In [7]:
### This agent responsible for performing the actual search on the web.###

# Taken from OpenAI Documentation
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."


search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required") # indicates that this agent is mandatory for appropriate running 
)

In [8]:
message = "Latest AI agent frameworks in 2025"

with trace("Basic Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of November 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing autonomous systems:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents and supports complex interaction logic. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes LLMs and mathematical frameworks to guide the design of adaptive AI agents. PCF enables real-time parameter reconfiguration through combinatorial spaces, allowing agents to dynamically adjust their behaviors. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system that introduces a standardized Agent-to-Agent communication layer based on the Model Context Protocol (MCP). It incorporates an Experience Pack architecture for structured knowledge retention and continual learning. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It focuses on curating high-quality training data and enhancing agent robustness through test-time reflection and voting. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework released in March 2025, focusing on creating multi-agent workflows with comprehensive tracing and guardrails. It is provider-agnostic, compatible with over 100 different LLMs. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Google Agent Development Kit (ADK)**: Announced in April 2025, this modular framework integrates with the Google ecosystem, including Gemini and Vertex AI. It supports hierarchical agent compositions and requires minimal code for efficient development. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Microsoft Semantic Kernel**: A lightweight framework bridging traditional development with AI capabilities, emphasizing seamless integration into enterprise applications. It supports multiple languages and orchestrates multi-step AI workflows. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **Microsoft AutoGen**: A flexible toolkit for developing advanced multi-agent systems, simplifying the building of conversational AI and task automation agents. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Technology, designed to independently execute complex real-world tasks without continuous human supervision. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

- **Kruti**: A multilingual AI agent and chatbot developed by Ola Krutrim, capable of performing real-world tasks for users by integrating directly with various online services. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

These frameworks represent the forefront of AI agent development, each contributing to the advancement of autonomous systems across various applications. 

In [9]:
### Planner agent - responsible for taking a query and execute several search agents who run based on that query in order to do deep reasearch.###

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"""You are a helpful research assistant. Given a query, come up with a set of web searches 
to perform to best answer the query. Output {HOW_MANY_SEARCHES} term to query for. """

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important th the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="Planner agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan
    )

In [10]:
message = "Latest AI agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find updated information on AI agent frameworks that are predicted or already established in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore predictions and trends in AI technologies, focusing on agent frameworks for 2025.', query='AI agent technology trends 2025'), WebSearchItem(reason='To identify specific frameworks that are emerging or innovative in the field of AI agents by 2025.', query='top AI agent frameworks 2025')]


In [11]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("yoharel.dev@gmail.com")  # Change to your verified sender
    to_email = To("yoharel.dev@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [12]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000025894ED7EC0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [13]:
### Email agent ###

INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini"
)

In [ ]:
### Writer agent responsible for writing the final research report ###

INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentences summary of the findings")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

writer_agent = Agent(
    name="Writer agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData
)


In [ ]:
""" flow: plan_searches -> perform_searches -> search """

async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query. 
        Result: WebSearchPlan 
    """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the plan.
    This actually perform the searches we have according to HOW_MANY_SEARCHES.
    We will run all the tasks in parallel and gether all results  
    """
    print("Searching...")
    num_completed = 0
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for seasrching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [ ]:
""" Housekeeping functions: one for the report, one for sending the email. """

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write the report based on the search results """
    print("Generating report based on the search results...")
    input = f"Original query: {query}\nSummarized seasrch results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [19]:
query = "Latest AI agents frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Research completed.")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Generating report based on the search results...
Finished writing report
Writing email...
Email sent
Research completed.
